#Space for allmy imports

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D,Input,BatchNormalization
from tensorflow.keras.utils import load_img

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Path to my collection of Images

In [ ]:
BASE_DIR='/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/UTKFace'

#Splitting the image label into age and gender and the imagepath

In [ ]:

image_paths=[]
age_labels=[]
gender_labels=[]

for filename in tqdm(os.listdir(BASE_DIR)):
    image_path=os.path.join(BASE_DIR,filename)
    temp=filename.split('_')
    age=int(temp[0])
    gender=int(temp[1])
    image_paths.append(image_path)
    age_labels.append(age)
    gender_labels.append(gender)

  0%|          | 0/23708 [00:00<?, ?it/s]

#Creation of a 3 column dataframe

In [ ]:
df=pd.DataFrame()
df['image'],df['age'],df['gender']=image_paths,age_labels,gender_labels

#Inspecting Data

In [ ]:
df.shape

(23708, 3)

In [ ]:
df.head()

,image,age,gender
0,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,80,0
1,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,80,0
2,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,80,0
3,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,80,0
4,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,80,1


In [ ]:
df.tail()

,image,age,gender
23703,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,15,0
23704,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,15,0
23705,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,15,1
23706,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,15,1
23707,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,15,1


In [ ]:
df.sample(10)

,image,age,gender
11809,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,34,0
13893,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,30,0
2947,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,69,1
14016,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,28,0
20371,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,21,0
19376,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,23,1
13862,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,30,0
21410,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,1,0
4620,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,55,0
11885,/content/drive/MyDrive/ISJ_ASSIGNMENT/DATASET/...,34,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23708 entries, 0 to 23707
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   23708 non-null  object
 1   age     23708 non-null  int64 
 2   gender  23708 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 555.8+ KB


In [ ]:
df.isnull().sum()

,0
image,0
age,0
gender,0


In [ ]:
round((df.isnull().sum()*100)/len(df),2)

,0
image,0.0
age,0.0
gender,0.0


In [ ]:
#map labels for gender
gender_dict={0:'Male',1:'Female'}

#Display one image from dataset

In [ ]:
from PIL import Image
img=Image.open(df['image'][0])
plt.axis('off')
plt.imshow(img)

Display the gender

In [ ]:
df['gender']

#Display images with their age and gender

In [ ]:
plt.figure(figsize=(25,25))
files=df.iloc[0:25]

for index,file,age,gender in files.itertuples():
    plt.subplot(5,5,index+1)
    img=load_img(file)
    img=np.array(img)
    plt.imshow(img)
    plt.title(f"Age:{age} Gender:{gender_dict[gender]}")
    plt.axis('off')

#method for extracting features from images

In [ ]:
def extract_features(images):
    features=[]

    for image in tqdm(images):
        img=Image.open(image).convert('L')
        img=img.resize((128,128),Image.ANTIALIAS)
        img=np.array(img)
        features.append(img)


    features=np.array(features)
    #skip if you are doing for rgb
    features=features.reshape(len(features),128,128,1)
    return features

In [ ]:
X=extract_features(df['image'])

  0%|          | 0/23708 [00:00<?, ?it/s]

#Analysing features

In [ ]:
X.shape

In [ ]:
X=X/255.0

In [ ]:
y_gender=np.array(df['gender'])
y_age=np.array(df['age'])

In [ ]:
input_shape=(128,128,1)

In [ ]:
#Create Model
input_shape = (128, 128, 1)
# declare input
inputs = Input((input_shape))
# Convolutional layers 1
conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu') (inputs)
maxp_1 = MaxPooling2D(pool_size=(2, 2)) (conv_1)
# Convolutional layers 2
conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu') (maxp_1)
maxp_2 = MaxPooling2D(pool_size=(2, 2)) (conv_2)
# Convolutional layers 3
conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu') (maxp_2)
maxp_3 = MaxPooling2D(pool_size=(2, 2)) (conv_3)
# Convolutional layers 4
conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu') (maxp_3)
maxp_4 = MaxPooling2D(pool_size=(2, 2)) (conv_4)


flatten = Flatten() (maxp_4)

# Fully connected layers
dense_1 = Dense(256, activation='relu') (flatten)
dense_2 = Dense(256, activation='relu') (flatten)

dropout_1 = Dropout(0.3) (dense_1)
dropout_2 = Dropout(0.3) (dense_2)

# two outputs
output_1 = Dense(1, activation='sigmoid', name='gender_out') (dropout_1)
output_2 = Dense(1, activation='relu', name='age_out') (dropout_2)

model = Model(inputs=[inputs], outputs=[output_1, output_2])

model.compile(loss=['binary_crossentropy', 'mae'], optimizer='adam', metrics={'gender_out': 'accuracy', 'age_out': 'mae'})

In [ ]:
from keras.utils import plot_model
plot_model(model)

In [ ]:
history=model.fit(x=X,y=[y_gender,y_age],batch_size=32,epochs=10,validation_split=0.2)

In [ ]:
#random number for get image
image_index = 1
print("Original Gender:", gender_dict[y_gender[image_index]], "Original Age:", y_age[image_index])
# predict from model
pred = model.predict(X[image_index].reshape(1, 128, 128, 1))
pred_gender = gender_dict[round(pred[0][0][0])]
pred_age = (pred[1][0][0])
print("Predicted Gender:", pred_gender, "Predicted Age:", pred_age)
plt.axis('off')
plt.imshow(X[image_index].reshape(128, 128), cmap='gray');